#### 5

Обучение дерева при 1 млн образцов занимает 1 час, сколько потребуется при 10 млн образцов?

In [1]:
import math
10 * math.log(10**7,2) / math.log(10**6,2)

11.666666666666668

#### 6

Обучить дерево

In [2]:
RS = 40

In [3]:
from sklearn.datasets import make_moons

X, y = make_moons(n_samples=10000, noise=0.4, random_state=RS)

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=RS)

In [5]:
list(range(1,10))

[1, 2, 3, 4, 5, 6, 7, 8, 9]

Подбираем параметры

In [6]:
%%time

from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
import warnings
warnings.simplefilter('ignore')

params = {
    'max_leaf_nodes': list(range(1,100)), 
    'splitter': ['best', 'random'], 
    'min_samples_split': list(range(1,5)), 
    'min_samples_leaf': list(range(1,5))
         }
grid_search_cv = GridSearchCV(DecisionTreeClassifier(random_state=RS), params, n_jobs=1, verbose=1, cv=3)

grid_search_cv.fit(X_train, y_train)

Fitting 3 folds for each of 3168 candidates, totalling 9504 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


CPU times: user 49.3 s, sys: 155 ms, total: 49.4 s
Wall time: 49.7 s


[Parallel(n_jobs=1)]: Done 9504 out of 9504 | elapsed:   49.5s finished


GridSearchCV(cv=3, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=40,
                                              splitter='best'),
             iid='deprecated', n_jobs=1,
             param_grid={'max_leaf_nodes': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11,
    

In [7]:
grid_search_cv.best_estimator_

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=4,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=40, splitter='best')

Оцениваем accuracy

In [8]:
from sklearn.metrics import accuracy_score

y_pred = grid_search_cv.predict(X_test)
accuracy_score(y_test, y_pred)

0.8493333333333334

#### 8

Создать лес из 100 деревьев

In [9]:
from sklearn.model_selection import ShuffleSplit # перемешивает индексы

n_trees = 1000 
n_instances = 100 # сколько образцов показать одному дереву

mini_sets = [] # сохраним сюда поднаборы для обучения каждого дерева

rs = ShuffleSplit(n_splits=n_trees, test_size=len(X_train) - n_instances, random_state=42)
for mini_train_index, mini_test_index in rs.split(X_train):
    X_mini_train = X_train[mini_train_index]
    y_mini_train = y_train[mini_train_index]
    mini_sets.append((X_mini_train, y_mini_train))

In [10]:
from sklearn.base import clone
# просто копируем дерево с лучшими параметрами
tree_ = clone(grid_search_cv.best_estimator_)

In [11]:
tree_

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=4,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=40, splitter='best')

In [12]:
mini_sets[0][0][0]

array([1.00118098, 1.03054651])

In [14]:
import numpy as np

forest = [tree_] * n_trees # список из 1000 одинаковых моделей

accuracy_scores = []

for tree, (X_mini_train, y_mini_train) in zip(forest, mini_sets):
    tree.fit(X_mini_train, y_mini_train)
    
    y_pred = tree.predict(X_test)
    accuracy_scores.append(accuracy_score(y_test, y_pred))

np.mean(accuracy_scores)

0.825016

In [20]:
np.max(accuracy_scores)

0.8526666666666667

In [23]:
X_test.shape

(1500, 2)

In [30]:
# создаем матрицу 1000*1500 
Y_pred = np.zeros([n_trees, X_test.shape[0]], dtype=np.int)
# записываем для каждого дерева вектор ответов для 1500 тестовых образцов
for tree_index, tree in enumerate(forest):
    Y_pred[tree_index] = tree.predict(X_test)

In [40]:
from scipy.stats import mode # возвращает моду - самый часто встречающийся вектор ответов

y_pred_mode = mode(Y_pred, axis=0)[0]

In [43]:
accuracy_score(y_test, y_pred_mode.reshape(-1))

0.844

Ура! Лес работает лучше.